# Automatic Centerlines Extraction


First, extract the coronary artery centerlines from your dataset using the method described in 
[Coronary artery centerline extraction in cardiac CT angiography using a CNN-based orientation classifier (Wolternik et al., 2019b)](https://www.sciencedirect.com/science/article/pii/S1361841518308491). For now, CCTA scans from `<dataset_dir>` are matched with the patterns `*.mhd`
and `*_image.nii.gz`. 

```bash
source scripts/centerlines_autoextract.sh <dataset_dir> <output_dir>
```

For every `datapoint_X` in `<dataset_dir>`, this command creates files `<output_dir>/<datapoint_X>/vessel0.txt`,
`<output_dir>/<datapoint_X>/vessel1.txt` etc., where each `vessel<N>.txt` file corresponds to a discovered coranary 
artery vessel and it contains an array of 4D points (3D XYZ coordinates + artery lumen radius estimate).

## Create the HD5-encoded Dataset

The HD5 file type allows on-the-fly loading of big datasets, without the need to load entire arrays in memory -- 
relevant parts are loaded as needed, reducing memory footprint. 

For more inforamation, refer to the 
[HD5 Python documentation](https://docs.h5py.org/en/stable/).

In [1]:
%cd ../

/home/marco/contrast-gan-3D


Adjust the variables in the following cell to match your setup:

In [2]:
from pathlib import Path

# DATASET_DIR = Path("/home/marco/data/ASOCA_Philips/images")
# DATASET_DIR = Path("/home/marco/data/MMWHS/ct_train")
DATASET_DIR = Path("/home/marco/data/MMWHS/ct_test")

OUTPUT_DIR = DATASET_DIR / "auto_centerlines"

# By default, .h5 converted files are saved inside `DATASET_DIR`; set this
# variable if you want to save them somewhere else
H5_OUTPUT_DIR = None

print(str(DATASET_DIR))
print(str(OUTPUT_DIR))
print(str(H5_OUTPUT_DIR))

/home/marco/data/MMWHS/ct_test
/home/marco/data/MMWHS/ct_test/auto_centerlines
None


In [3]:
from pprint import pprint

cctas = sorted(
    list(DATASET_DIR.glob("*.mhd")) + list(DATASET_DIR.glob("*_image.nii.gz"))
)
centerlines = sorted([d for d in OUTPUT_DIR.glob("*") if d.is_dir()])
print(f"Found {len(cctas)} CCTAs with {len(centerlines)} centerlines")

zipped = list(zip(cctas, centerlines))
pprint(zipped[:3])

Found 40 CCTAs with 40 centerlines
[(PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2001_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2001_image')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2002_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2002_image')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2003_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2003_image'))]


In [4]:
from contrast_gan_3D.utils import io_utils, logging_utils

logging_utils.set_project_loggers_level(level="DEBUG")

contrast_gan_3D.utils.geometry: INFO -> DEBUG
contrast_gan_3D.utils.io_utils: INFO -> DEBUG


In [5]:
for ccta, centerlines_dir in zipped:
    if io_utils.stem(centerlines_dir) == io_utils.stem(ccta):
        io_utils.sitk_to_h5(
            ccta,
            centerlines_dir,
            centerlines_dir / "ostia.xml",
            h5_output_dir=H5_OUTPUT_DIR,
        )
        print()
    else:
        print(f"**No centerlines for {str(ccta)!r}**")

[2024-03-13 17:34:52,712: INFO] Changed orientation '/home/marco/data/MMWHS/ct_test/ct_test_2001_image.nii.gz': LAS -> LPS (contrast_gan_3D.utils.io_utils:80)
[2024-03-13 17:34:52,997: DEBUG] Original image dtype float32 range (-3022.0, 1931.0) (contrast_gan_3D.utils.io_utils:90)
[2024-03-13 17:34:53,214: DEBUG] New image dtype int16 range (-1024, 1500) (contrast_gan_3D.utils.io_utils:102)
[2024-03-13 17:34:53,248: DEBUG] CCTA: (512, 512, 224) centerlines: (6029, 4) ostia: (2, 3) (contrast_gan_3D.utils.io_utils:138)
[2024-03-13 17:34:53,249: DEBUG] H5 file: '/home/roel/data/MMWHS/ct_test/ct_test_2001_image.h5' (contrast_gan_3D.utils.io_utils:147)

[2024-03-13 17:34:56,355: INFO] Changed orientation '/home/marco/data/MMWHS/ct_test/ct_test_2002_image.nii.gz': LAS -> LPS (contrast_gan_3D.utils.io_utils:80)
[2024-03-13 17:34:56,572: DEBUG] Original image dtype float32 range (-3022.0, 1386.0) (contrast_gan_3D.utils.io_utils:90)
[2024-03-13 17:34:56,759: DEBUG] New image dtype int16 range (-

In [1]:
# import matplotlib.pyplot as plt
# import torch
# from torchvision.utils import make_grid

# from contrast_gan_3D.data.HD5Scan import HD5Scan
# from contrast_gan_3D.utils import geometry as geom
# from contrast_gan_3D.utils.visualization import plot_mid_slice

# fname = (cctas[0].parent / io_utils.stem(cctas[0])).with_suffix(".h5")
# print(fname)
# with HD5Scan(fname) as scan:
#     im = scan.ccta[::].astype(float)
#     labs = scan.labelmap[::]

# # HWD -> CDHW
# grid_mat = im[..., ::6].transpose(2, 0, 1)[:, None]
# grid = make_grid(torch.from_numpy(grid_mat), normalize=True, value_range=(-260, 740))

# # HWD -> CDHW
# labs_mat = labs[..., ::6].transpose(2, 0, 1)[:, None]
# grid_labs = make_grid(torch.from_numpy(labs_mat))
# cart = geom.grid_to_cartesian_coords(grid_labs.squeeze().permute(1, 2, 0))

# fig, ax = plt.subplots(figsize=(15, 15))

# ax.imshow(grid.permute(1, 2, 0))
# # HWD : yxz
# ax.scatter(cart[:, 1], cart[:, 0], c="red", s=plt.rcParams["lines.markersize"] * 0.8)
# ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

# plt.tight_layout()
# plt.show()
# plt.close(fig)

# fig, axes = plt.subplots(1, 3, figsize=(12, 6))

# plot_mid_slice(im, axes=axes, vmin=-260, vmax=740)

# plt.tight_layout()
# plt.show()
# plt.close(fig)